# Install tfds-nightly

This will prevent a checksum error when downloading the dataset later

In [ ]:
!pip install tfds-nightly

In [ ]:
import tensorflow as tf

# Unzip the model

The model should be in /mymodel.zip
It will be unzipped to /tmp/mymodel


In [ ]:
from zipfile import ZipFile
saved_model_path = "/mymodel.zip"

# Create a ZipFile Object and load sample.zip in it
with ZipFile(saved_model_path, 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall("/")


In [ ]:
student_model = tf.saved_model.load("/tmp/mymodel/1")
infer = student_model.signatures['serving_default']

## Import the Data

This will download the oxford flowers dataset from TFDS
We will create test examples from the last 90%

In [ ]:
import tensorflow_datasets as tfds
splits = ['train[:80%]', 'train[80%:90%]', 'train[90%:]']

(train_examples, validation_examples, test_examples), info = tfds.load('oxford_flowers102', with_info=True, as_supervised=True, split = splits)

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [ ]:
# Mapping function to resize the images to 224x224
def format_image(image, label):
  image = tf.image.resize(image, (224,224)) / 255.0
  return  image, label

test_batches = test_examples.map(format_image).batch(1)

In [ ]:
# Here we will perform the testing. It's only taking the top class, but
# this dataset has 102 classes, so often the correct class will be #2 or #3
# As such we should give a relatively generous passmark, so I chose 60
# The model in my colab gets 69%

eval_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='eval_accuracy')
for images, labels in test_batches:
  predictions = infer(images)['output_1']
  eval_accuracy(labels, predictions)

print(eval_accuracy.result())
score = ((eval_accuracy.result()*100).numpy())

if score>60:
  print("Passed")
else:
  print("Failed")


